# Chapter 7 데이터 정제 및 준비 

- 해당 장에서는 결측치, 중복 데이터, 문자열 처리 그리고 다른 분석적 데이터 변환에 대한 도구들을 다룬다.

## 7.1 누락된 데이터 처리하기

- pandas의 설계 목표 중 하나는 누락 데이터를 가능한 한 쉽게 처리할 수 있도록 하는 것.
- pandas 객체의 모든 기술 통계는 누락된 데이터를 배제하고 처리함.
- pandas는 누락된 데이터를 실숫값인 NaN으로 취급함.

In [2]:
import pandas as pd
import numpy as np

In [3]:
string_data = pd.Series(['aardvark','artichoke',np.nan,'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

> 파이썬의 내장 None 값 또한 NA 값으로 취급됨.

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

#### NA 처리 메서드

<details>
<summary>NA 처리 메서드</summary>
<div markdown="1">

|인자|설명|
|:--|:--|
|dropna|누락된 데이터가 있는 축(로우,컬럼)을 제외시킨다. 어느 정도의 누락 데이터까지 용인할 것인지 지정할 수 있다.|
|fillna|누락된 데이터를 대신할 값을 채우거나 'ffill'이나 'bfill' 같은 보간 메서드를 적용한다.|
|isnull|누락되거나 NA인 값을 알려주는 불리언값이 저장된 같은 형의 객체를 반환한다.|
|notnull|isnull과 반대되는 메서드|
    
</div>
</details>

### 7.1.1 누락된 데이터 골라내기 (dropna)

> dropna를 사용하면 매우 간편하게 누락된 데이터를 골라낼 수 있다.  
> Series에 dropna 메서드를 적용하면 $\text{널이 아닌}^{\text{non-null}}$ 데이터와 색인값만 들어 있는 Series를 반환한다.

In [9]:
from numpy import nan as NA

data = pd.Series([1,NA,3.5,NA,7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [10]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

> pandas.isnull, pandas.notnull이나 불리언 색인을 사용해 직접 손으로 제거할 수도 있다.

In [11]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

> DataFrame 객체의 경우 NA를 제거하는 두 가지 방법이 있다.  
> - 별도의 지정이 없는 경우 : NA 값을 하나라도 포함하고 있는 로우나 컬럼을 제외한다.  
> - how='all' 옵션을 넘기면 모두 NA 값인 로우만 제외시킨다.

In [12]:
data = pd.DataFrame([[1.,6.5,3],[1.,NA,NA],
                     [NA,NA,NA],[NA,6.5,3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [13]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [14]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


> 컬럼을 제외시키는 방법도 동일하게 동작한다.  
> 옵션으로 axis = 1을 넘겨주면 된다.

In [16]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [20]:
data.dropna(how='all', axis = 'columns')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


> DataFrame의 로우를 제외시키는 방법은 시계열 데이터에 주로 사용되는 경향이 있음.  
> 몇 개 이상의 값이 들어 있는 로우만 살펴보고 싶다면 thresh 인자에 원하는 값을 넘기면 된다.

In [22]:
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,1] = NA
df.iloc[:2,2] = NA

df

,0,1,2
0,-0.704895,NaN,NaN
1,0.429249,NaN,NaN
2,-1.356953,NaN,1.603586
3,0.286830,NaN,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


In [23]:
df.dropna()

,0,1,2
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


In [24]:
df.dropna(thresh=2)

,0,1,2
2,-1.356953,NaN,1.603586
3,0.286830,NaN,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


### 7.1.2 결측치 채우기 (fillna)

> 누락된 값을 제외시키지 않고 데이터 상의 '구멍'을 어떻게든 메우고 싶은 경우 fillna 메서드에 채워 넣고 싶은 값을 넘겨주면 된다.

In [26]:
df.fillna(0)

,0,1,2
0,-0.704895,0.000000,0.000000
1,0.429249,0.000000,0.000000
2,-1.356953,0.000000,1.603586
3,0.286830,0.000000,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


> fillna에 사전값을 넘겨서 각 컬럼마다 다른 값을 채울 수도 있다.

In [27]:
df.fillna({1:0.5,2:0})

,0,1,2
0,-0.704895,0.500000,0.000000
1,0.429249,0.500000,0.000000
2,-1.356953,0.500000,1.603586
3,0.286830,0.500000,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


> fillna는 `새로운 객체를 반환`하지만 inplace=True를 사용하면 `기존 객체를 변경`할 수도 있다.

In [29]:
_ = df.fillna(0, inplace = True)
df

,0,1,2
0,-0.704895,0.000000,0.000000
1,0.429249,0.000000,0.000000
2,-1.356953,0.000000,1.603586
3,0.286830,0.000000,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


> 재색인에서 사용 가능한 보간 메서드는 fillna 메서드에서도 사용 가능하다.  
> 예를 들어 method = "ffill", "bfill"

In [31]:
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:,1] = NA
df.iloc[4:,2] = NA
df

,0,1,2
0,-0.982547,1.002809,1.119510
1,-0.434269,1.717283,0.730459
2,-0.615606,NaN,-1.687928
3,-0.407083,NaN,0.399892
4,0.285687,NaN,NaN
5,-1.777159,NaN,NaN


In [32]:
df.fillna(method = 'ffill')

,0,1,2
0,-0.982547,1.002809,1.119510
1,-0.434269,1.717283,0.730459
2,-0.615606,1.717283,-1.687928
3,-0.407083,1.717283,0.399892
4,0.285687,1.717283,0.399892
5,-1.777159,1.717283,0.399892


In [33]:
df.fillna(method='ffill', limit = 2)

,0,1,2
0,-0.982547,1.002809,1.119510
1,-0.434269,1.717283,0.730459
2,-0.615606,1.717283,-1.687928
3,-0.407083,1.717283,0.399892
4,0.285687,NaN,0.399892
5,-1.777159,NaN,0.399892


> 조금만 창의적으로 생각하면 fillna를 이용해서 매우 다양한 일을 할 수 있다.  
> 예를 들면 Series의 평균값이나 중간값을 전달할 수도 있다.

In [35]:
data = pd.Series([1.,NA,3.5,NA,7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

#### fillna 함수 인자

<details>
<summary>fillna 함수 인자</summary>
<div markdown="1">

|인자|설명|
|:--|:--|
|value|비어 있는 값을 채울 스칼라값이나 사전 형식의 객체|
|method|보간 방식. 기본적으로 'ffill'을 사용한다.|
|axis|값을 채워 넣을 축. 기본값은 axis = 0이다.|
|inplace|복사본을 생성하지 않고 호출한 객체를 변경한다. 기본값은 False다.|
|limit|값을 앞 혹은 뒤에서부터 몇 개까지 채울지 지정한다.|
    
</div>
</details>

## 7.2 데이터 변형

### 7.2.1 중복 제거하기

> DataFrame의 duplicated 메서드는 각 로우가 중복인지 아닌지 알려주는 불리언 Series를 반환한다.  
> drop_duplicates는 duplicated 배열이 False인 DataFrame을 반환한다.

In [39]:
data = pd.DataFrame({'k1': ['one','two'] * 3 + ['two'],
                     'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [40]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [42]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


> 두 메서드는 기본적으로 모든 컬럼에 적용되며 중복을 찾아내기 위한 부분합을 따로 지정해 줄 수도 있다.  
> 새로운 컬럼을 하나 추가하고 'k1' 컬럼에 기반해서 중복을 걸러내려면 다음과 같이 한다.

In [48]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


> duplicated와 drop_duplicates는 기본적으로 처음 발견된 값을 유지함.  
> keep='last' 옵션을 넘기면 마지막으로 발견된 값을 반환한다.

In [49]:
data.drop_duplicates(['k1','k2'], keep = 'last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 7.2.2 함수나 매핑을 이용해서 데이터 변형하기

In [50]:
data = pd.DataFrame({'food' : ['bacon','pulled pork','bacon','Pastrami','corned beef','Bacon','pastrami','honey ham','nova lox'],
                     'ounces' : [4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


> 해당 육류가 어떤 동물의 고기인지 알려줄 수 있는 컬럼을 추가한다고 가정, 육류별 동물을 담고 있는 사전 데이터를 아래와 같이 작성

In [52]:
meat_to_animal = {
    'bacon' : 'pig',
    'pulled pork' : 'pig',
    'pastrami' : 'cow',
    'corned beef' : 'cow',
    'honey ham' : 'pig',
    'nova lox' : 'salmon'
}

> Series의 map 메서드는 사전류의 객체나 어떤 함수를 받을 수 있다.  
> 우선, 육류 일므에 대소문자가 섞여 있는 사소한 문제가 있으므로 str.lower 메서드를 사용해서 모두 소문자로 변경한다.

In [54]:
lowercased = data['food'].str.lower()

In [55]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [60]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


> 함수를 넘겨서 같은 일을 수행할 수도 있다.

In [61]:
data['food'].map(lambda x : meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 7.2.3 값 치환하기

> fillna 메서드는 누락된 값을 채우는 작업은 일반적인 값 치환 작업.  
> map 메서드를 한 객체 안에서 값의 부분집합을 변경하는 데 사용했다면 replace 메서드는 같은 작업에 대해 좀 더 간단하고 유연한 방법을 제공함.

In [ ]:
data = pd.Series()